In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # 讀取 .env 檔案
api_key = os.getenv("GOOGLE_API_KEY")
Mongo_url = os.getenv("MONGO_URI")
Database_Name = os.getenv("Database_Name")
Collection_Name = os.getenv("Collection_Name")

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "eveng": {
            "transport": "streamable_http",
            "url": "http://127.0.0.1:4200/my-custom-path/"
        },
    }
)

tools = await client.get_tools()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash-preview-05-20",
    temperature=1.0,
    max_retries=2,
    google_api_key=api_key,
)

model = llm.bind_tools(tools)

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.mongodb.aio import AsyncMongoDBSaver
from langmem.short_term import SummarizationNode
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.messages.utils import count_tokens_approximately
from typing import Any
import time

async with AsyncMongoDBSaver.from_conn_string(Mongo_url) as checkpointer:

	summarization_node = SummarizationNode( 
    token_counter=count_tokens_approximately,
    model=model,
    max_tokens=1000000,
    max_summary_tokens=2048,    # 摘要預算
    max_tokens_before_summary=80000,  # 觸發摘要的令牌數
    output_messages_key="llm_input_messages",
)

	class State(AgentState):
		# NOTE: we're adding this key to keep track of previous summary information
		# to make sure we're not summarizing on every LLM call
		context: dict[str, Any]
	
	agent = create_react_agent(
		model = model,
		tools = tools,
		checkpointer = checkpointer,
		pre_model_hook=summarization_node, 
		state_schema=State,
		prompt="""當用戶請你幫忙解析影片時，請使用download(MCP tool)下載影片，
				當影片超過300秒(5分鐘)時使用audio_cut(MCP tool)依每5分鐘的長度分割成多個片段，
				再使用whisper_tool(MCP tool)產生字幕檔並根據回傳的文字進行影片內容解析，請用繁體中文回答，
				切記在使用MCP工具時一定要先詢問我並告知你預計傳入的參數"""
	)

	config = {
		"configurable": {
			"thread_id": "1"  
		}
	}

	async def main():
		while True:
			user_input = input()
			if user_input == "exit":
				return
			events = agent.astream(
				{"messages": user_input},
				config,
				stream_mode = "values",
			)
			output = []
			async for event in events:
				output.append(event["messages"][-1].pretty_repr(True))

			for line in output[-2:]:
				print(line)
			time.sleep(0.2)

	await main()

	state = await agent.aget_state(config)
	print(state)
	# print(state.values["messages"][-1].content)


In [ ]:
# from pymongo import AsyncMongoClient
# async def main():
#     client = AsyncMongoClient(Mongo_url)
#     try:
#         await client.drop_database("checkpointing_db")
#     except Exception as e:
#         raise Exception("Unable to find the document due to the following error: ", e)
# await main()